In [ ]:
class Customer:
    '''
- Customers are checked in an out of rooms
- customers leave feedback depending on how their stay was
- i.e. they are successfully checked in, room is clean
- Customers are happy if room is clearn
- Customers are unhappy if room is overbook or unclean'''
    def __init__(self, bookedRoom, name):
        '''class constructor'''
        self.__bookedRoom = bookedRoom
        self.__name = name
        self.__feedback =0
        self.__rating = 0 #added a rating
    
    '''Methods'''   
    def incrementRating (self, increment):
        '''setter for incrementing rating  / hotel experience'''
        self.__rating +=increment
        
    def getName(self):
        '''Returns the customer's name'''
        return self.__name
    def getRoom(self):
        '''Returns the booked room'''
        return self.__bookedRoom
    def getRating(self)
        '''Returns the rating'''
        return self.__rating